In [10]:
import flywheel
import os
import datetime
from dateutil.tz import tzutc

fw = flywheel.Client()
collection_id = '60536423b2b486911f893843'
# gear = fw.lookup('gears/antsct-aging-fw')
gear_name = 'wscore-ct-heatmap'
local_dir = '/home/will/Desktop/wscore_testing'
os.makedirs(local_dir, exist_ok=True)

In [11]:
# Get all the sessions within the collection.
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [12]:
for session in sessions:
    sub_label = fw.get(session.parents['subject']).label.replace('_','x')
    ses_dir = os.path.join(local_dir, sub_label, session.label)
    os.makedirs(ses_dir, exist_ok=True)
    
    for analysis in session.analyses:
        if 'wscore_2021-04-01' in analysis.label and analysis.job['state']=='complete' and analysis.files:
            file1 = [f for f in analysis.files if f.name.endswith('.zip')]
            file2 = [f for f in analysis.files if f.name.endswith('wscores.csv')]
            file3 = [f for f in analysis.files if f.name.endswith('.nii.gz')]
            
            try:
                analysis.download_file(file1[0].name, os.path.join(ses_dir, file1[0].name))
                analysis.download_file(file2[0].name, os.path.join(ses_dir, file2[0].name))
                analysis.download_file(file3[0].name, os.path.join(ses_dir, file3[0].name))
            except IndexError as e:
                print(session.label)